In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC, NuSVC
import tensorflow.keras as keras
import csv
import time

In [2]:
X = []
Y = []

with open('./new_data.csv', newline='') as csfile:
    spamreader = csv.reader(csfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        X.append(row[0])
        Y.append(int(row[1]))

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
def get_data(X):
    data = []
    for x in X:
        img = cv2.imread(x)
        img = cv2.resize(img, (128, 128))
        data.append(img)

    return data

In [8]:
X_train_arr = get_data(X_train)

In [ ]:
def compute_descriptors(X):
    list_des = []
    list_kp = []
    kaze = cv2.KAZE_create()
    for x in X:
        img = cv2.imread(x, 0)
        img = cv2.resize(img, (128, 128))
        kp, des = kaze.detectAndCompute(img, None)
        list_des.append(des)
        list_kp.append(kp)
    return list_kp, list_des

In [ ]:
list_kp, list_des = compute_descriptors(X_train)
for i in range(len(list_des)):
    t = np.sum(list_des[i] == None)
    if t == 1:
        list_des[i] = np.zeros((64, 64))
print(np.mean([i.shape[0] for i in list_des]))


In [ ]:
def get_points(list_kp):
    res = []
    for kp in list_kp:
        m = len(kp)
        points = np.zeros((m, 2))
        for i in np.arange(m):
            points[i] = np.array(kp[i].pt)

        res.append(points)

    return res

In [ ]:
def get_train_date(list_kp, n=8):
    m = len(list_kp)
    X_train = np.zeros((m, n))
    kmeans = KMeans(n_clusters=n, random_state=0)
    for i in np.arange(m):
        if len(list_kp[i]) < n:
            X[i] = np.ones((1,n)) if len(list_kp[i]) != 0 else np.zeros((1,n))
        else:
            kmeans.fit(list_kp[i])
            histogram = np.histogram(kmeans.labels_, bins=n)
            X_train[i] = histogram[0] / np.sum(histogram[0])
    return X_train

In [ ]:
list_points = get_points(list_kp)
train = get_train_date(list_points)

In [ ]:
kaze = cv2.KAZE_create()
img = cv2.imread('./img/001.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = cv2.resize(img, (128, 128))
gray = cv2.resize(gray, (128, 128))
s = time.time()
kp, des = kaze.detectAndCompute(gray, None)
print(time.time() - s)

In [ ]:
class_id = [i.class_id for i in kp]
pt = [i.pt for i in kp]
my_pt = [(i.angle, i.size) for i in kp]

In [ ]:
pt_arr = np.array(pt)

In [ ]:
mask = np.array(class_id) == 1

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0)
kmeans.fit(pt_arr)
labels = kmeans.labels_
histogram = np.histogram(labels, bins=8)

In [ ]:
kp_array = np.array(kp)
key = kp_array[labels == 7]
key = list(key)

In [ ]:
img_kp = cv2.drawKeypoints(gray, key, img)
plt.imshow(img_kp)

In [13]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = keras.layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)


    return X

In [14]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X


    # First component of main path
    X = keras.layers.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = keras.layers.Activation('relu')(X)


    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X_shortcut, X])
    X = keras.layers.Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [17]:
def model(input_shape = (64, 64, 3), classes = 6):

    # Define the input as a tensor with shape input_shape
    X_input = keras.layers.Input(input_shape)


    # Zero-Padding
    X = keras.layers.ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = keras.layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = 'b')
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = 'c')
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = 'd')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'b')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'c')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'd')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'e')
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'b')
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = keras.layers.AveragePooling2D(pool_size = (2, 2), name='avg_pool')(X)

    ### END CODE HERE ###

    # output layer
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)


    # Create model
    model = keras.Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [18]:
model = model(input_shape=(128,128,3), classes=2)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 134, 134, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
X_train_arr = np.array(X_train_arr)
Y_train_arr = np.zeros((len(Y_train), 2))

for i in range(len(Y_train)):
    Y_train_arr[i, Y[i]] = 1

In [ ]:
model.fit(X_train_arr, Y_train_arr, epochs=52, batch_size=64, shuffle=True)

Epoch 1/52
 21/127 [===>..........................] - ETA: 9:16 - loss: 3.0605 - accuracy: 0.7984 

In [ ]:
list_kp_test, list_des_test = compute_descriptors(X_test)
for i in range(len(list_des_test)):
    t = np.sum(list_des_test[i] == None)
    if t == 1:
        list_des_test[i] = np.zeros((64, 64))

In [ ]:
list_points_test = get_points(list_kp_test)
test = get_train_date(list_points_test)

In [ ]:
X_test_ar = test[:,np.newaxis, :]

In [ ]:
y_pred = model.predict(X_test_ar)
y_pred_target = np.argmax(y_pred, axis=1)
y_pred_probability = np.max(y_pred, axis=1)

In [ ]:
y_target = np.array(Y_test)
recall = recall_score(y_target, y_pred_target)
precission = precision_score(y_target, y_pred_target)
f1 = f1_score(y_target, y_pred_target)

Y_test_ar = np.zeros((len(test), 2))
for i in range(len(test)):
    Y_test_ar[i, Y_test[i]] = 1
roc_auc = roc_auc_score(Y_test_ar, y_pred)

In [ ]:
print(f'accuracy: {accuracy_score(y_target, y_pred_target)}')
print(f'recall: {recall}')
print(f'precission: {precission}')
print(f'f1: {f1}')
print(f'roc_auc: {roc_auc}')

In [ ]:
print(f'accuracy: {accuracy_score(y_target, y_pred_target)}')
print(f'confusion_matrix: \n{confusion_matrix(y_target, y_pred_target)}')
print(f'roc_auc: {roc_auc}')

In [ ]:
## X_train

In [ ]:
y_pred = model.predict(X_train_ar)
y_pred_tar = np.argmax(y_pred, axis=1)

In [ ]:
y_target = np.array(Y_train)
recall = recall_score(y_target, y_pred)
precission = precision_score(y_target, y_pred)
f1 = f1_score(y_target, y_pred)
roc_auc = roc_auc_score(y_target, y_pred)

In [ ]:
print(f'recall: {recall}')
print(f'precission: {precission}')
print(f'f1: {f1}')
print(f'roc_auc: {roc_auc}')


In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')


